In [ ]:
import os

import pandas as pd

In [ ]:

in_relative_path = "../exports_covoit/"
in_absolute_path = os.path.abspath(os.path.join(os.getcwd(), in_relative_path))
print(in_absolute_path)

In [ ]:
os.listdir(in_absolute_path)

In [ ]:
df = pd.read_csv(f"{in_absolute_path}/2023-06.csv", sep=";", engine="python")
df.head()

Chaque ligne correspond à un trajet de covoiturage, c"est à dire un couple passager / conducteur. A chaque passager est donc affecté un trajet.

Exemple : un conducteur réalise un déplacement avec deux passagers différents au sein de son véhicule, le nombre de trajets réalisés et de 2. Ceci se traduit par deux lignes.

Colonnes à garder :  
- journey_id  
Identifiant unique affecté à un couple passager/conducteur.  
- trip_id  
Identifiant permettant de regrouper plusieurs couples passager/conducteur dans un même véhicule.  
Exemple : c4124bb1-d8a4-487c-b4d9-367b931ee8ce  
- journey_start_datetime  
Exemple : 2019-10-31T23:00:00.000Z  
Date et heure du départ au format ISO 8601 (YYYY-MM-DDThh:mm:ssZ).  
L"heure est exprimée en UTC (Coordinated Universal Time). UTC n"est pas ajusté sur l"heure d"été et hiver !  
- journey_start_insee  
Exemple : 91377  
Code INSEE commune ou arrondissement du point de départ (prise en charge passager). Déterminé via les coordonnées géographiques  
- journey_start_town  
Exemple : Massy  
Commune du point de départ (prise en charge passager). Déterminé via les coordonnées géographiques  
- journey_end_insee  
Exemple : 91122  
Code INSEE commune ou arrondissement du point d"arrivée (dépose passager).Déterminé via les coordonnées géographiques  
- journey_end_town  
Exemple : Bures-sur-Yvette  
Commune du point d"arrivée (dépose passager). Déterminé via les coordonnées géographiques  
- passenger_seats  
Exemple : 2  
Nombre de sièges réservés par l"occupant passager  
- journey_distance  
Exemple: 34523  
Distance parcourue en mètres. Information envoyée par l"opérateur  
- journey_duration  
Exemple: 60  
Durée du trajet en minutes. Information envoyée par l"opérateur  

Colonnes à supprimer : 
- journey_start_date
Exemple : 2019-10-31
Date départ au format YYYY-MM-DD
- journey_start_time
Exemple : 02:20:001
Heure de départ au format hh:mm:ss
- journey_start_lat
Exemple : 48.725
Latitude du point de départ (prise en charge passager) comprise entre 90deg et -90deg décimaux en datum WSG-84
- journey_start_lon
Exemple : 2.261
Longitude du point de départ (prise en charge passager) comprise entre 180deg et -180deg décimaux en datum WSG-84
- journey_start_postcode Supprimé à partir de Juin 2022
Exemple : 91300
Code postal du point de départ (prise en charge passager). Déterminé via les coordonnées géographiques
- journey_start_towngroup
Exemple : Ile-De-France Mobilites
EPCI ou AOM de départ. Déterminé via les coordonnées géographiques. Peut être vide
- journey_start_department
Exemple : 38
Département du point de départ (prise en charge passager). Déterminé via les coordonnées géographiques
- journey_start_country
Exemple : France
Pays du point de départ (prise en charge passager). Déterminé via les coordonnées géographiques
- journey_end_datetime
Exemple : 2019-10-31T23:15:00.000Z
Date et heure de l"arrivée au format ISO 8601 (YYYY-MM-DDThh:mm:ssZ).
L"heure est exprimée en UTC (Coordinated Universal Time). UTC n"est pas ajusté sur l"heure d"été et hiver !
- journey_end_date
Exemple : 2019-10-31
Date d"arrivée au format YYYY-MM-DD
- journey_end_time
Exemple : 02:20:001
Heure d"arrivéet au format hh:mm:ss
- journey_end_lat
Exemple : 48.695
Latitude du point d"arrivée (dépose passager) comprise entre 90deg et -90deg décimaux en datum WSG-84.
- journey_end_lon
Exemple : 2.162
Longitude du point d"arrivée (dépose passager) comprise entre 180deg et -180deg décimaux en datum WSG-84
- journey_end_postcode Supprimé à partir de Juin 2022
Exemple : 91440
Code postal du point d"arrivée (dépose passager). Déterminé via les coordonnées géographiques
- journey_end_department
Exemple : 38
Département du point d"arrivée (dépose passager). Déterminé via les coordonnées géographiques
- journey_end_towngroup
Exemple : Ile-De-France Mobilites
EPCI ou AOM d"arrivée. Déterminé via les coordonnées géographiques. Peut être vide
- journey_end_country
Exemple : France
Pays du point d"arrivée (dépose passager)
- operator_class
Exemple: C
Classe de preuve de covoiturage
- has_incentive
Exemple: OUI
Si le trajet est incité ou non par un opérateur ou une campagne paramétrée sur le registre. Attention : à date cette donnée est renseignée partiellement et ne peut donner lieu à des analyses et conclusions.

In [ ]:
colonne_a_garder = [
    "journey_id",
    "trip_id",
    "journey_start_date",
    "journey_start_insee",
    "journey_start_town",
    "journey_end_insee",
    "journey_end_town",
    "passenger_seats",
    "journey_distance",
    "journey_duration"
]
df = df[colonne_a_garder]
df.sample(5)

In [ ]:
df.describe()

In [ ]:
df.describe(include="object")

In [ ]:
df["passenger_seats"].value_counts()

Il y a une très grande majoritée de réservation de covoiturage avec un seul passager.

In [ ]:
df[df["journey_start_town"].isna()]["journey_start_insee"].value_counts()

Les quelques données manquantes en "journey_start_town" et "journey_end_town" correspondent à trajets avec des villes dans un pays étranger.  
Majoritairement Monaco (Code INSEE 99138), la Suisse (Code INSEE 99140) et l"Italie (Code INSEE 99127) pour juin 2022.

In [ ]:
df.groupby(["journey_start_date"])["passenger_seats"].sum().reset_index()


In [ ]:
import numpy as np
df["journey_distance_tot_passengers"] = (df["journey_distance"] * df["passenger_seats"])
df.groupby("journey_start_date").agg(
    nb_passagers=("passenger_seats", np.sum),
    nb_trajets=("trip_id", "nunique"),
    total_distance=("journey_distance_tot_passengers", np.sum)).reset_index()

Logiquement, il y a très peu de différence entre le nombre de trajets et le nombre de passagers. Comme vu précédemment, 98% des réservations concernent un seul passager.